# Process hourly files

In [ ]:
import glob
import pandas as pd
import os

In [ ]:
data_folders_path = '/Volumes/Seagate/misc/historical_data'
destination_path = '/Volumes/Seagate/misc'

data_folders = glob.glob(data_folders_path + "/2023*")
data_folders.sort()
print(data_folders)

In [ ]:
cols_to_keep = ['detecttime', 'hash', 'status', 'curblocknumber', 'blockspending', 'timepending', 'gas', 'gasprice', 'toaddress', 'fromaddress', 'maxpriorityfeepergas', 'maxfeepergas','basefeepergas', 'gasused']

first_file = True
for folder in data_folders:

    hourly_files = glob.glob(folder + '/*.csv.gz')
    hourly_files.sort()
    for hour in hourly_files:
        print(hour)
        
        # Load the data
        df = pd.read_csv(hour, compression='gzip', sep='\t')
        print(df['detecttime'].iloc[0])

        # Filter for us-east-1 and type 2 txns
        df = df[df['region'] == 'us-east-1']
        df = df[df['type'] == 2]

        # Keep only the columns we want
        df = df[cols_to_keep]

        # append the df to a master df
        master_file = os.path.join(destination_path, 'master.csv.gz')
        if first_file == True:
            df.to_csv(master_file, compression='gzip', sep='\t', mode='a', header=True, index=False)
            first_file = False
        else:
            df.to_csv(master_file, compression='gzip', sep='\t', mode='a', header=False, index=False)

In [ ]:
# open the master file. Is the column detecttime monotonically increasing? If not there is a problem
master_file = '/Volumes/Seagate/misc/master.csv.gz'

df = pd.read_csv(master_file, compression='gzip', sep='\t')
df['detecttime'].is_monotonic_increasing